In [ ]:
import json
import openai

In [ ]:
api_key = 'Your API key'
openai.api_key = api_key

In [ ]:
upload_response = openai.File.create(
  file=open('1000.jsonl', "rb"),
  purpose='fine-tune'
)
upload_response
file_id = upload_response.id
file_id

In [ ]:
upload_response = openai.File.create(
  file=open('pre_done/Validation_n.jsonl', "rb"),
  purpose='fine-tune'
)
upload_response
file_id2 = upload_response.id
file_id2

In [ ]:
# Start training and wait. Be aware of costs before starting.
response = openai.FineTuningJob.create(training_file=file_id, model="babbage-002",
                                       hyperparameters={'n_epochs':6})
job_id = response["id"]
status = response["status"]
print(f'Fine-tunning model with jobID: {job_id}.')
print(f"Training Response: {response}")
print(f"Training Status: {status}")

In [ ]:
#Can be used to monitor the status
status = openai.FineTuningJob.list_events(id=job_id, limit=5)
status

In [ ]:
#Try out a single response
completion = openai.ChatCompletion.create(
  model="the number of your trained model",
  messages=[
    {"role": "system", "content": "Write a very concise procedure given the reactants and products, esterification reaction. Use one word actions and precise temperatures and durations. Skip measurements."},
    {"role": "user", "content": "CCO.O=C(O)c1ccccc1-c1ccc(C(F)(F)F)cc1>>O=C([O-])c1ccccc1-c1ccc(C(F)(F)F)cc1 ->"}
  ],
  stop=[" \n"],
  max_tokens=100,
  temperature=0.05)
print(completion.choices[0].message)

In [ ]:
#Multiple queries for the entire Validation or Testing dataset. Be aware of the costs.
#Can take a while if the servers are experiencing a heavy load.

In [ ]:
from tqdm import tqdm
import time

In [ ]:
#For 3.5-turbo
with open('pre_done/Testing.jsonl', 'r') as file:
    lines = file.readlines()

predictions = []
for line in tqdm(lines):
    time.sleep(3)
    entry = json.loads(line)
    
    new_prompt = entry['prompt']
    
    completion = openai.ChatCompletion.create(
  model="the number of your trained model",
  messages=[
    {"role": "system", "content": "Write a very concise procedure given the reactants and products, esterification reaction. Use one word actions and precise temperatures and durations. Skip measurements."},
    {"role": "user", "content": new_prompt}
  ],
  stop=[" \n"],
  max_tokens=100,
  temperature=0.05)
    
    prediction = completion.choices[0].message['content'][1:]
    predictions.append(prediction)

# Step 3: Write the predictions to a new text file
with open('file_name_to_save.txt', 'w') as file:
    for prediction in predictions:
        file.write(f"{prediction}\n".replace('\u00b0',''))

In [ ]:
#For davinci and babbage
from tqdm import tqdm
with open('pre_done/Testing.jsonl', 'r') as file:
    lines = file.readlines()

# Step 2: Loop through each entry, extract the prompt, and get a prediction
predictions = []
for line in tqdm(lines):
    time.sleep(2)
    entry = json.loads(line)
    #print(entry)
    
    new_prompt = entry['prompt']
   
    answer = openai.Completion.create(
        model='the number of your trained model',
        prompt=new_prompt,
        max_tokens=120,
        stop=[" \n"],
         temperature=0.05
    )

    
    prediction = answer['choices'][0]['text'][1:]
    predictions.append(prediction)


with open('file_name_to_save.txt', 'w') as file:
    for prediction in predictions:
        file.write(f"{prediction}\n".replace('\u00b0',''))

In [ ]:
#Calculate BLEU from a file

import json
import sacrebleu

prediction_file_name = 'path to your file with predictions'

with open('pre_done/Testing.jsonl', 'r') as file:
    lines = file.readlines()
    correct_answers = [json.loads(line)['completion'] for line in lines]

with open(prediction_file_name, 'r') as file:
    predictions = [line.strip() for line in file.readlines()]

correct_answers_str = [' '.join(ans.split()) for ans in correct_answers]
predictions_str = [' '.join(pred.split()) for pred in predictions]

bleu = sacrebleu.corpus_bleu(predictions_str, [correct_answers_str])
print(f'BLEU score: {bleu.score:.4f}')
print(f'Individual n-gram precisions: {bleu.precisions}')